# Fine-tuning the model

Although our model is quite general for different time span, observational strategies, and colors. It can imporve to better performance on a specific mission or to strengthen the model's ability for dealing with some special cases.

Thus, we supply an interface for you to fine-tune the model on given datasets. In this tutorial, we will show how to use the fine-tunning interface. Here we choose some ZTF archive data as an example.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

make sure you have GPU

In [ ]:
torch.cuda.set_device(0) # change this if you have more than one GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

make sure the training is deterministic and reproducible

In [ ]:
import random
seed = 42

random.seed(seed)  # python random generator
np.random.seed(seed)  # numpy random generator

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    import random

    random.seed(worker_seed)


g = torch.Generator()
g.manual_seed(seed)

## Load your own dataset

Your own dataset should located at a directory as the following structure:

```
dataset
├── train
│   ├── 0
│   ├── 1
│   └── ...
└── test
    ├── 0
    ├── 1
    └── ...
└── train_labels.csv
└── test_labels.csv
```

All the data inside the train/test folder can be any format that can be constructed as numpy ndarrays.

In our work, we only use light curve as input, so the data are all light curve ndarrays. But you can also include parameters since we have parameter component in our combined network. Other components like spectrum/SED are not implenmented in our model, but can be easily added.

You might need to modify the following code to load your own dataset properly.

In [ ]:
# First, load your dataset to lists of numpy ndarrays:

train_lcs = []
train_params = []
train_labels = []

test_lcs = []
test_params = []
test_labels = []

train_csv = np.loadtxt('dataset/train_labels.csv', delimiter=',')
test_csv = np.loadtxt('dataset/test_labels.csv', delimiter=',')

# If light curves are not normalized to relative variability, please apply our 
# `light_curve_preparation` function to them first

for i in range(len(train_csv)):
    lc_id = int(train_csv[i][0])
    train_lcs.append(np.load(f'dataset/train/{lc_id}.npy'))
    train_params.append(train_csv[i][2:])
    train_labels.append(train_csv[i][1])
    
    test_lcs.append(np.load(f'dataset/test/{lc_id}.npy'))
    test_params.append(test_csv[i][2:])
    test_labels.append(test_csv[i][1])


Then, send the data to our dataset object:

In [ ]:
from deep_lc import LocalDataset

In [ ]:
training_set = LocalDataset(train_lcs, train_params, train_labels)
test_set = LocalDataset(test_lcs, test_params, test_labels)

## Modify hyperparameters

In [ ]:
from deep_lc.config import PROPOSAL_NUM, BATCH_SIZE, LR, LABELS

Change above imported hyperparameters to your own hyperparameters. For example, you can change the number of epochs, the learning rate, the batch size, and labels etc.

In [ ]:
hyper_parameters={
    "batch_size": BATCH_SIZE,
    "lr": LR/10, # lower learning rate for fine-tuning
    "weight_decay": 0,
    "labels": LABELS,
    "proposal_num": PROPOSAL_NUM,
},


## Load baseline model

Here I only show how to fine-tune the combined model, since it's the most powerful model. For other models, you may need to implement the fine-tuning interface by yourself.

In [ ]:
model_dict = torch.load("combined_17.pth", map_location=device)

## Fine-tune the model

There are lots of strageties for fine-tuning. Here we use the simplest one: fine-tune the whole model with a small learning rate. In pratices, you may need to try different learning rate, different optimizer, different loss function, and different fine-tuning strageties for different datasets. For example, smaller but similar datasets may only need to fine-tune the last few layers while larger or more different datasets may need to fine-tune the whole model with a smaller learning rate.

In [ ]:
from deep_lc import finetune

finetune(
    training_set=training_set,
    test_set=test_set,
    hyper_params=hyper_parameters,
    base_model=model_dict,
    save_dir=".",
)


## Calibration

We provide a conformal prediction algorithm to calibrate the model